In [1]:
import numpy as np
import pandas as pd
import streamlit as st
import csv
from collections import defaultdict
from numpy.random import default_rng as rng

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from os import path
from matplotlib.pyplot import figure
if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

from rsi_modules import *

mm3_url         = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=1475905714"
publication_url = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=2125519370"
theses_url      = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=609352385"

F = pd.read_csv(mm3_url)
publication = pd.read_csv(publication_url).fillna("")
theses = pd.read_csv(theses_url).fillna("")
theses = theses.drop_duplicates(subset=['รหัส (นักศึกษา)'], keep='last')

In [2]:
def MAIN_report_mm3(F):
    def calculate_teaching_hours_per_year(courses_year):
        for yr in courses_year.keys():  
            ajarn_teach_hr = defaultdict(float)
            for name_hr in courses_year[yr]:
                for name, hr in name_hr.items():
                    ajarn_teach_hr[name]+=hr
            courses_year[yr]=ajarn_teach_hr
        TEACH_TERM = pd.DataFrame(courses_year).fillna(0).reset_index()
        TEACH_TERM.rename(columns={'index':'name'}, inplace=True)
        hr_courses_year = MEMBERS.merge(TEACH_TERM, on='name', how='left').fillna(0)
        return hr_courses_year

    def identify_course_code_owner(course_owners):
        course_code_owner = defaultdict(list)
        for n,cw in zip(course_owners['name'],course_owners['owner_courses']):
            course_code = defaultdict(list)
            for c in cw:
                course_code[c[1]].append(c[0])
            course_code_owner[n] = course_code
        course_code_owner = pd.DataFrame(course_code_owner).T.fillna('0').reset_index().rename(columns={'index':'name'})
        course_code_owner = MEMBERS.merge(course_code_owner, on='name', how='left').fillna(0)
        return course_code_owner

    def count_owner_courses(course_code_owner):
        number_course_code_owner = defaultdict(list)
        for c in course_code_owner.columns:
            if c != 'name':
                for row in course_code_owner[c]:
                    if row != 0:
                        number_course_code_owner[c].append(len(row))
                    else:
                        number_course_code_owner[c].append(0)
                # number_course_code_owner[c] = course_code_owner[c]
        number_course_code_owner = pd.DataFrame(number_course_code_owner)
        number_course_code_owner['name']= course_code_owner['name']

        Member_Courses = MEMBERS.merge(number_course_code_owner, on='name', how='left').fillna(0)
        return Member_Courses
    

    # Main function
    f = remove_empty_columns(F)
    f = merge_multiple_academic_years(f)
    _, courses_year = courses(f)
    names = defaultdict(list)
    for aj,co,yr in zip(f['อาจารย์ผู้รับผิดชอบรายวิชา'],f['รหัสวิชา'],f['academic_year']):
        names[aj].append((co,yr))

    MEMBERS = pd.DataFrame(faculty_members()).rename(columns={0:'name'})
    course_owners = pd.DataFrame([names]).T.rename(columns={0:'owner_courses'}).reset_index().rename(columns={'index':'name'})

    # หาค่า ชั่วโมงสอน
    hr_courses_year = calculate_teaching_hours_per_year(courses_year)
    # หาค่า เจ้าของรหัสวิชา
    course_code_owner = identify_course_code_owner(course_owners)
    # หาค่า จำนวนรหัสวิชาที่เป็นเจ้าของ
    number_course_code_owner = count_owner_courses(course_code_owner)

    return hr_courses_year, course_code_owner, number_course_code_owner
    

In [3]:
F = pd.read_csv(mm3_url)
hr_courses_year, course_code_owner, number_course_code_owner = MAIN_report_mm3(F)

/workspaces/academic-staff-workload/rsi_modules.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f.dropna(how='all', axis=1, inplace=True)
/workspaces/academic-staff-workload/rsi_modules.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['academic_year'] = expand.values()


In [4]:
academic_yrs = ['[2568]-เทอม 1','[2568]-เทอม 2','[2567]-เทอม 1','[2567]-เทอม 2']

for i, ay in enumerate(academic_yrs):
    one_semester = pd.DataFrame()
    one_semester['name'] = hr_courses_year['name']
    one_semester['จำนวนชั่วโมงสอน'] = hr_courses_year[ay].round(2)
    one_semester['เจ้าของรหัสวิชา'] = course_code_owner[ay]
    one_semester['จำนวนรหัสวิชาที่เป็นเจ้าของ'] = number_course_code_owner[ay]
    one_semester.to_csv(f'teaching/teaching_{ay}.csv', index=False)

In [5]:
b = pd.DataFrame()
for i, ay in enumerate(['[2568]-เทอม 1','[2568]-เทอม 2']):
    a = pd.read_csv(f'teaching/teaching_{ay}.csv')
    b[i]=a['จำนวนชั่วโมงสอน']
b = pd.DataFrame(b.sum(axis=1)).rename(columns={0:'จำนวนชั่วโมงสอน'})
b['name'] = a['name']
c = pd.DataFrame()
c['name'] = b['name']
c['จำนวนชั่วโมงสอน'] = b['จำนวนชั่วโมงสอน']
c.to_csv('teaching/teaching_total.csv', index=False)

In [6]:
b

,จำนวนชั่วโมงสอน,name
0,0.00,นพ.สมเกียรติ ลีละศิธร
1,7.00,พญ.วัชรา ริ้วไพบูลย์
2,63.50,ศ.ดร.ทวี เชื้อสุวรรณทวี
3,91.00,รศ.ดร.อาดัม นีละไพจิตร
4,69.00,รศ.ดร.ณัฏฐนียา โตรักษา
5,85.00,ผศ.ดร.อารี ภาวสุทธิไพศิฐ
6,87.00,ผศ.ดร.ธีรศักดิ์ ศรีสุรกุล
7,88.83,ผศ.ดร.เจนจิรา เจนจิตรวาณิช
8,83.83,ดร.รติรส จันทรสมดี
9,84.33,ดร.อิศวรา ศิริรุ่งเรือง


In [6]:
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
df.to_csv('publication.csv')
# st.write(df)

2026-02-18 04:28:00.847 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-18 04:28:00.981 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-18 04:28:00.981 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-18 04:28:00.982 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [5]:
df

,ปี,,Q1,Q2,Q3,Q4,TCI1,TCI2,อื่นๆ
0,2564 (2021),0,0,0,0,0,1,0,0
1,2565 (2022),0,0,0,0,0,0,2,0
2,2566 (2023),0,0,0,0,0,4,0,0
3,2567 (2024),2,2,1,0,0,3,0,1
4,2568 (2025),1,0,2,4,2,1,0,3
5,2569 (2026),0,2,0,0,0,0,0,0


In [7]:
def classify_publication(publication):  

    NAMES = []
    
    for n in publication.columns:
        if 'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ' in n:
            NAMES.append(n) 

    for t in publication.columns:
        if 'วารสารไทย (TCI1-TCI2)' in t: publication.rename(columns={t:'วารสารไทย'}, inplace=True)
        elif 'วารสารนานาชาติ (Q1-Q4 หรืออื่นๆ)' in t: publication.rename(columns={t:'วารสารนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับนานาชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับชาติ'}, inplace=True)

    name_paper = defaultdict(list)
    for name in NAMES:
        papers = []
        for i,(role,TCI,Q,conf_int,conf_nat) in enumerate(zip(publication[name],
                                                            publication['วารสารไทย'],
                                                            publication['วารสารนานาชาติ'],
                                                            publication['ประชุมวิชาการระดับนานาชาติ'],
                                                            publication['ประชุมวิชาการระดับชาติ'])):
            if role in ['First Author', 'Corresponding Author']:
                papers+=[TCI+Q+conf_int+conf_nat]
        name_paper[name.replace('ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [','').replace(']','')] = papers

    categories = ['Q1','Q2','Q3','Q4','TCI1','TCI2','อื่นๆ']
    blanktable = pd.DataFrame(np.zeros((len(name_paper.keys()), len(categories)),dtype=int), columns=categories)
    blanktable['name']=name_paper.keys()


    for idx, (name, papers) in enumerate(name_paper.items()):
        for paper in papers:
            for cat in categories:
                if cat in paper:
                    blanktable.loc[idx, cat] += 1
    
    # Reorder columns with 'name' first
    blanktable = blanktable[['name'] + [col for col in blanktable.columns if col != 'name']]
    
    # blanktable.to_csv('publication_classified.csv')

    return blanktable

In [9]:
def get_publications_by_author(publication):
    """
    Create a dictionary where:
    - Key: author name (First Author or Corresponding Author)
    - Value: list of tuples (year, title, publication_type)
    """
    
    # Find author columns
    NAMES = []
    for n in publication.columns:
        if 'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ' in n:
            NAMES.append(n)
    
    # Rename journal/conference columns for easier processing
    pub_copy = publication.copy()
    for t in pub_copy.columns:
        if 'วารสารไทย (TCI1-TCI2)' in t: 
            pub_copy.rename(columns={t:'วารสารไทย'}, inplace=True)
        elif 'วารสารนานาชาติ (Q1-Q4 หรืออื่นๆ)' in t: 
            pub_copy.rename(columns={t:'วารสารนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับนานาชาติ' in t: 
            pub_copy.rename(columns={t:'ประชุมวิชาการระดับนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับชาติ' in t: 
            pub_copy.rename(columns={t:'ประชุมวิชาการระดับชาติ'}, inplace=True)
    
    author_publications = defaultdict(list)
    
    # Get year and title columns
    year_col = 'ปีที่ตีพิมพ์ พศ. (คศ.)'
    title_col = 'ชื่อเรื่อง'
    
    # Process each row
    for idx, row in pub_copy.iterrows():
        year = row.get(year_col, '')
        title = row.get(title_col, '')
        
        # Determine publication type (Q or TCI category)
        pub_type = 'อื่นๆ'
        for category in ['Q1', 'Q2', 'Q3', 'Q4', 'TCI1', 'TCI2']:
            combined_text = (str(row.get('วารสารไทย', '')) + 
                           str(row.get('วารสารนานาชาติ', '')) +
                           str(row.get('ประชุมวิชาการระดับนานาชาติ', '')) +
                           str(row.get('ประชุมวิชาการระดับชาติ', '')))
            if category in combined_text:
                pub_type = category
                break
        
        # Process each author
        for name_col in NAMES:
            role = row.get(name_col, '')
            if role in ['First Author', 'Corresponding Author','EIC']:
                if role == 'First Author': role = 'FA'
                elif role == 'Corresponding Author': role = 'CA'
                # Extract author name from column name
                author_name = name_col.replace('ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [', '').replace(']', '')
                # Add publication tuple (year, title, type)
                author_publications[author_name].append((year, pub_type, role, title))
    
    return dict(author_publications)


# Export author_pubs to CSV
def export_author_publications_to_csv(author_pubs, filename):
    """
    Export author publications dictionary to CSV file
    Format: Author, Year, Publication Type, Title
    """
    rows = []
    for author, publications in author_pubs.items():
        for year, pub_type, role, title in publications:
            rows.append({
                'Author': author,
                'Year': year,
                'Publication Type': pub_type,
                'Role': role,
                'Title': title
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Exported {len(rows)} publications to {filename}")
    return df

# Test the function
author_pubs = get_publications_by_author(publication)
df_export = export_author_publications_to_csv(author_pubs, 'author_publications.csv')


Exported 31 publications to author_publications.csv


In [10]:
author_pubs

{'ดร.วิษณุ นิตยธรรมกุล': [('2568 (2025)',
   'Q2',
   'CA',
   'Using Virtual Reality and Conversational AI to Enhance High School Students’ Intentions to Pursue Computer Science: Insights from the CS RMUTT Virtual Tour'),
  ('2568 (2025)',
   'Q3',
   'CA',
   'Development of Low-Cost PLC Training Kit for Enhancing Industry-Relevant Skills and Certification: PLC-CertX'),
  ('2568 (2025)',
   'Q3',
   'EIC',
   'The 3D-POD Model for AI-Driven Institutional Transformation and Graduate Employment Readiness in Thailand in Digital Era'),
  ('2568 (2025)',
   'Q4',
   'EIC',
   'Online Learner Achievement and Satisfaction on the ThaiMOOC Platform: A Case Study of Digital Media Creation on Online Social Networks'),
  ('2568 (2025)',
   'Q3',
   'EIC',
   'Conversational AI-Powered VR Development Model for Tourism Promotion in Thailand: Expert Assessment and Stakeholder Acceptance'),
  ('2568 (2025)',
   'อื่นๆ',
   'CA',
   'Design of the System Architecture of Competency-Based MOOC Using Mi

In [11]:

# Export to CSV
df_export.head()


,Author,Year,Publication Type,Role,Title
0,ดร.วิษณุ นิตยธรรมกุล,2568 (2025),Q2,CA,Using Virtual Reality and Conversational AI to...
1,ดร.วิษณุ นิตยธรรมกุล,2568 (2025),Q3,CA,Development of Low-Cost PLC Training Kit for E...
2,ดร.วิษณุ นิตยธรรมกุล,2568 (2025),Q3,EIC,The 3D-POD Model for AI-Driven Institutional T...
3,ดร.วิษณุ นิตยธรรมกุล,2568 (2025),Q4,EIC,Online Learner Achievement and Satisfaction on...
4,ดร.วิษณุ นิตยธรรมกุล,2568 (2025),Q3,EIC,Conversational AI-Powered VR Development Model...


In [ ]:
st.title('ภาระงานของคณาจารย์ในสถาบันราชสุดาปีการศึกษา 2568')

st.subheader("1. ชั่วโมงสอนและจำนวนวิชาที่รับผิดชอบ")

academic_year = st.radio(
    "เลือกเทอมและปีการศึกษา",
    ["2568/1", "2568/2"],
)

if academic_year == "2568/1":
    start_date = '8/5/2025'
    end_date   = '9/19/2025'
elif academic_year == "2568/2":
    start_date = '12/12/2025'
    end_date   = '12/30/2025'

f = F[F['Timestamp'].between(start_date, end_date)]
t1,t2 = report_mm3(f)

st.subheader("2. ผลงานตีพิมพ์ทางวิชาการในแต่ละปี")
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
st.write(df)

st.subheader("3. ภาระงานควบคุมวิทยานิพนธ์")
B = pd.DataFrame(get_thesis_workload_new(theses)).T
st.write(B)

st.subheader("4. ภาพรวมความก้าวหน้าวิทยานิพนธ์ของนักศึกษา")
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)
st.write(A)
st.write("หมายเหตุ:")
st.write("MA  = หส.ปริญญาโท สาขาคุณภาพชชีวิตคนพิการ")
st.write("MEd = หส.ปริญญาโท สาขาการศึกษาพิเศษ")
st.write("PhD = หส.ปรัชญาดุษฎีบัณฑิต สาขาคุณภาพชีวิตคนพิการ")

In [ ]:
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)

In [ ]:
st.write(A)